In [15]:
# We'll start with our library imports...
from __future__ import print_function
from tensorflow import keras 
import numpy as np                 # to use numpy arrays
import tensorflow as tf            # to specify and run computation graphs
import tensorflow_datasets as tfds # to load training data
import matplotlib.pyplot as plt    # to visualize data and draw plots
from tqdm import tqdm              # to track progress of loops
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer, Dense, Flatten, Conv2D, MaxPool2D, MaxPooling2D, BatchNormalization, Activation, Dropout, GlobalAvgPool2D

In [40]:
# load the text dataset
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:90%]', 'train[10%:]', 'test'),
    as_supervised=True)

x_input = train_data.shuffle(10000).batch(1300)
x_validation = validation_data.batch(1300)


In [41]:
# The mapping is very important
MAX_SEQ_LEN = 200
MAX_TOKENS = 5000
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens = MAX_TOKENS, 
    output_mode='int',
    output_sequence_length = MAX_SEQ_LEN)
train_data = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_data)

In [42]:
inputLayer = tf.keras.layers.Input(shape = (), dtype = tf.string, name = "input")

embedding_layer = tf.keras.layers.Embedding(input_dim = 10000, 
                                            output_dim = 512, 
                                            mask_zero = True)

BIDI1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True))
BIDI2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True))
GlobalPool = tf.keras.layers.GlobalMaxPooling1D()
LSTM1 = tf.keras.layers.LSTM(512, return_sequences = True)
LSTM2 = tf.keras.layers.LSTM(512, return_sequences = True)
gru1 = tf.keras.layers.GRU(256, return_sequences = True) 
gru2 = tf.keras.layers.GRU(128)
layer128 = tf.keras.layers.Dense(128, activation=tf.nn.swish)

layer64 = tf.keras.layers.Dense(64, activation=tf.nn.swish)

layer32 = tf.keras.layers.Dense(32, activation=tf.nn.swish)

output_layer = tf.keras.layers.Dense(1, activation="sigmoid")

model1 = tf.keras.Sequential([
    inputLayer, 
    vectorize_layer, 
    embedding_layer,
    BIDI1,
    LSTM1,
    gru1,
    GlobalPool,
    layer128,
    Dropout(0.5),
    layer64,
    Dropout(0.25),
    layer32,
    output_layer
])

model1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [43]:
history = model1.fit(x = x_input,
                    epochs=40,
                    validation_data=x_validation,
                    verbose=1)

Epoch 1/40


CancelledError:  [_Derived_]RecvAsync is cancelled.
	 [[{{node gradient_tape/sequential_7/embedding_7/embedding_lookup/Reshape/_76}}]] [Op:__inference_train_function_902401]

Function call stack:
train_function


In [ ]:
testing = test_data.batch(1300)
model1.evaluate(testing)

In [23]:
tf.saved_model.save(model1, '/work/cse479/izzatadly/HW2Model/Comp1IMDB')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /work/cse479/izzatadly/HW2Model/Comp1IMDB/assets


INFO:tensorflow:Assets written to: /work/cse479/izzatadly/HW2Model/Comp1IMDB/assets
